In [ ]:
import pandas as pd
import nltk
import re
import numpy as np

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
path = input('Введите путь к файлу ')
testdata = open(path, 'r')

In [ ]:
test = pd.read_csv(testdata)
test = test.loc[test['role'] != 'client']
test = test.reset_index(drop=True)
length = len(test) - 1

In [ ]:
test['text'] = test.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)

In [ ]:
chat_flag = pd.unique(test['dlg_id'])
chatborders = list()

for flag in chat_flag:
    number = test[test['dlg_id'] == flag].index.values.astype(int)
    chatborders.append(number[0])

chatborders.append(length)

# Ищу границы диалогов

In [ ]:
test['text'] = pd.Series(test['text'], dtype = "string")

In [ ]:
test['marked'] = ''

In [ ]:
greetings_list = ['здравствуйте', 'добрый', 'день', 'вечер', 'доброе', 'утро', 'приветствую', 'привет']
goodbyes_list = ['свидания','встречи', 'пока', 'прощай', 'доброго']
my_name_is = ['зовут', 'это']
company_namings = ['компания', 'компанию', 'компании']

In [ ]:
managers_greetings = {a: '' for a in range(len(chatborders) - 1)}
managers_company = {a: '' for a in range(len(chatborders) - 1)}
managers_names = {a: '' for a in range(len(chatborders) - 1)}
# Генерирую словари по границам диалогов

In [ ]:
def marker(lst, data, border, uptol, downtol):
    for line in lst:
        if test['marked'].iloc[border + uptol:border + downtol].any() == True:
            break
        test['marked'].iloc[border + uptol:border + downtol] = test['text'].iloc[border + uptol:border + downtol].str.contains(line, case=False, regex=False)
# Для проверки условия "менеджер поздоровался и попрощался, отмечаю все границы диалогов плюс допуск. Uptol - uptolerance и downtol - downtolerance - коэффиценты насколько можно отходить от границ диалогов"

In [ ]:
for border in chatborders[:-1]:
    marker(greetings_list, test, border, 0, 3)

In [ ]:
for border in chatborders[1:-1]:
    border += 1
    marker(goodbyes_list, test, border, -4, -1)

In [ ]:
marker(goodbyes_list, test, length, 0, 0)

In [ ]:
test['marked'] = test['marked'].map({True:1, False:0})

In [ ]:
chatborders1 = str(chatborders)
x = 0
for index in range((len(chatborders1))):
    chat1 = test['marked'].iloc[chatborders[index]:chatborders[index + 1]]
    result = chat1.sum()
    lst = list()
    if result >= 2:
        lst.append(True)
    else:
        lst.append(False)
    phrases_ind = list((chat1.loc[chat1 == 1]).index)
    for ind in phrases_ind:
        phrase = str(test.loc[ind]['text'])
        phrase = re.sub("[.,!']", "", phrase)
        lst.append(phrase)
    managers_greetings[x] = lst
    if index == 5:
        break
    x += 1

# Проверяю условие, записываю в список значение и записываю туда же фразы приветсвия и прощания. Потом список становится значением словаря

In [ ]:
test['to be extracted'] = ''

In [ ]:
def extractor(lst, data, border, uptol, downtol):
    for line in lst:
        if test['to be extracted'].iloc[border + uptol:border + downtol].any() == True:
            break
        ex_lst = test['text'].iloc[border + uptol:border + downtol].str.contains(line, case=False, regex=False)
        index1 = list((ex_lst.loc[ex_lst == True]).index)
        return index1, line
# Функция для "вытаскивания" индексов нужных ячеек и ключевых слов в этих ячейках

In [ ]:
def name_func(index1, line, uptol, downtol, POS):
    n_list = list()
    n_phrase = str(test['text'][index1])
    n_phrase1 = re.sub("[.,!']", "", n_phrase)
    n_list.append(n_phrase1)
    n_phrase1 = n_phrase1.split()
    n_index = n_phrase1.index(line)
    name = n_phrase1[n_index + uptol:n_index + downtol]
    from nltk import pos_tag
    name_pos = dict(pos_tag(name))
    name1 = list()
    for key in name_pos:
        if name_pos.get(key) == POS:
            name1.append(key)
            
    n_list.append(name1)
    return n_list

# Функция для анализа содержимого ячеек и выбора необходимых слов по значению части речи. Развернутый комментарий по этой функции в конце

In [ ]:
x = 0
for border in chatborders[:]:
    index1, line = extractor(company_namings, test, border, 0, 3)
    if len(index1) == 0:
        managers_company[x] = None
    else:
        company1 = name_func(index1, line, 0, 4, 'NNP')
        managers_company[x] = company1
    x += 1
# Ищу названия компаний

In [ ]:
test['to be extracted'] = ''

In [ ]:
x = 0
for border in chatborders[:-1]:
    index1, line = extractor(my_name_is, test, border, 0, 3)
    if len(index1) == 0:
        managers_names[x] = None
    else:
        name1 = name_func(index1, line, -1, 2, 'NN')
        managers_names[x] = name1
    x += 1
# Ищу названия имена менеджеров

In [ ]:
for x in range(len(chatborders) - 1):
    name_manager = managers_names[x]
    print('РАЗГОВОР ' + str(x + 1))
    if name_manager is None:
        print('Менеджер разговора ' + str(x + 1) + ' не представился')
    else: 
        print('В разговоре ' + str(x + 1) + ' учавствовал менеджер ' + str(name_manager[-1]) + ', о чем было сообщено вот так: ' + str(name_manager[0]))
    comp = managers_company[x]
    
    if comp is None:
        print('Менеджер разговора ' + str(x + 1) + ' не назвал компанию')
    else:
        print('Менеджер разговора назвал компанию ' + str(comp[-1]))
    greet = managers_greetings[x]
    if greet[0] == True:
        print('Менеджер разговора ' + str(x + 1) + ' поздоровалась и попрощался(ась), умничка! Вот как это было: ' + '\n' + str(greet[1:]))
    elif (greet[0] == False and len(greet) != 1):
        print('Менеджер разговора ' + str(x + 1) + ' не совсем вежлив:' + '\n' + str(greet[1]) + '\nИ это все, менеджер разговора ' + str(x + 1) + '?:(')
    else:
        print('Менеджер разговора ' + str(x + 1) + ' совсем невежливо себя ведет! Где же привет и пока?')

In [ ]:
# По результатам работы name_func можно отметить следующее:
# (одного менеджера зовут "компания", а фирма называется "китобизнес удобно")
# Функция POS-tagging в NLTK очень слабая. Я рассчитывала, что она хотя бы сможет показывать верно глаголы и 
# местоимения, чтобы по ним можно было отделить от других сущностей, пусть она бы ошиблась в, например, "диджитал бизнес"
# Но она оказалась совсем слабой, и пришлось намеренно, при поиске имен, вытаскивать слова с частью речи NN, хотя,
# по идее, это должно было быть NNP. 
# Вероятно, подошел бы другой POS-tagging, хотя лучший, что я видела, считался SOTA и был успешен лишь в 60% случаев.
# Самый надежный вариант - самый топорный - собрать изначально названия всех компаний и имена менеджеров в отдельный набор
# и по ним уже искать.